# New Helios++ Python API

This is the SSC write-up of the current state of the new Python API. It is not intended as a final, user-facing documentation. Instead, it is a response to the API mockup produced by the Helios developers in spring 2024. We have not implemented your write-up 1:1 but used it guide our design. Reasons to deviate are either technical or because we see an even better alternative.

## Loading and running surveys from XML (Use Case 1)

Most user-facing objects can be import directly from the `helios` package:

In [ ]:
import helios
import numpy as np

Construction of objects is done via `classmethod`s - allowing multiple such methods:

In [ ]:
survey = helios.Survey.from_xml("./data/surveys/toyblocks/tls_toyblocks.xml")

The survey object exposes many interesting linked things as properties (like `scanner`, `scene`, `platform`, `legs` etc.). In the use case of loading from XML this is not super interesting yet:

In [ ]:
survey.scanner

We can run the survey by using the `run()` method. It accepts both `execution_settings` and `output_settings` as parameters. The execution setting combine everything related to the *execution* of the survey, so these parameters do not modify the results of the computation. Instead they modify how exactly that results is calculated. Examples of execution settings are e.g. number of threads, parallelization strategtes, logging or KDTree options. You can instantiate these execution settings as objects, if you want to pass them around. The `ExecutionSettings` class, like all other classes in the API, has `dataclass`-like behaviour. So you can pass parameters to `__init__` or set them as properties later:

In [ ]:
exec_settings = helios.ExecutionSettings(num_threads=1)

In [ ]:
exec_settings.num_threads = 2

Also, we enforce `pydantic`-based validation on all properties of all objects in the new API. This leads to early errors for malformed input:

In [ ]:
exec_settings.num_threads = 0

In [ ]:
exec_settings = None

Similar to `ExecutionSettings`, the `OutputSettings` class controls how the generated point cloud is stored and/or returned to the user. Its most important parameter is `format` whose values are a string enum `OutputFormat`. Possible values are `OutputFormat.LAS`, `OutputFormat.LAZ`, `OutputFormat.XYZ`, `OutputFormat.NPY` and `OutputFormat.LASPY`.

In [ ]:
output_settings = helios.OutputSettings(format=helios.OutputFormat.NPY)

Our instances of `execution_settings` and `output_settings` can now be passed to `survey.run`:

In [ ]:
points, trajectory = survey.run(
    execution_settings=exec_settings, output_settings=output_settings
)

For convenience, you can also pass individual settings directly into `survey.run`. This makes changing one or two parameters much more readable, but also allows you to organize settings into reusable objects:

In [ ]:
points, trajectory = survey.run(format=helios.OutputFormat.NPY)

With the output format being `OutputFormat.NPY`, we receive a `numpy` array of structured dtype containing all the columns that we have available on the C++ side:

In [ ]:
points

This allows us to use square bracket indexing to extract a column like this:

In [ ]:
points["position"]

In [ ]:
points["intensity"]

The `trajectories` bit of the return gives trajectory information for ALS, but returns `None` for TLS. For `OutputFormat.NPY` and `OutputFormat.LASPY`, we receive an in-memory representation of the point cloud. For the file-based output formats, we instead receive the directory, where the data has been written:

In [ ]:
output = survey.run(format=helios.OutputFormat.XYZ, output_dir="demo-output")

In [ ]:
output

If we look at an ALS survey, we can see that also the trajectory data is returned as a structured dtype array:

In [ ]:
survey = helios.Survey.from_xml("./data/surveys/toyblocks/als_toyblocks.xml")

In [ ]:
points, trajectory = survey.run(format=helios.OutputFormat.NPY)

In [ ]:
trajectory

Again, we can extract columns from that using square brackets:

In [ ]:
trajectory["gps_time"]

**TODOs aus Use Case 1**:
* FullWaveForm settings
* Ensure that all output and execution settings are actually functional

## Compose survey in Python with XML-loaded components (Use Case 2)

All relevant objects behave similar to `Survey` in the sense that they can be loaded with `from_xml` and have their properties modified (and have them validated). For scanners and platforms, whose XML definitions are packaged with Helios++, we also provide functions in the `helios.scanner` and `helios.platforms` modules that directly instantiate the relevant scanners:

In [ ]:
scene = helios.StaticScene.from_xml("data/scenes/demo/box_scene.xml")

In [ ]:
from helios.scanner import riegl_vz_400
from helios.platforms import tripod

In [ ]:
scanner = riegl_vz_400()
platform = tripod()

With these objects, we can go ahead and define the survey object:

In [ ]:
survey = helios.Survey(scanner=scanner, platform=platform, scene=scene)

We could have also added legs directly here via the property `legs`, but there is a convenience method `add_leg` on the survey for that. It takes either a pre-instantiated `helios.Leg` object or `scanner_settings` and `platform_settings`. These settings work very similar to `Survey.run`'s settings in the sense that you can either pass them as an object or individually - what ever produces more readable code for you:

In [ ]:
scanner_settings = helios.ScannerSettings(pulse_frequency=2000, scan_frequency=200)

To solve any incompatibilities from properties with units, we implemented inputs with attached units based on the Python package `pint`. Units are however, resolved and converted on the input validation layer. Pint unit expressions can either be expressed as strings or by multiplying with units from the `helios.units` object. So for example you could use the following:

In [ ]:
scanner_settings.rotation_start_angle = "0 deg"
scanner_settings.rotation_stop_angle = 10 * helios.units.deg
scanner_settings.head_rotation = "10 deg/s"

We could also define `PlatformSettings` as an object, but here we feed the relevant bits directly into `survey.add_leg`:

In [ ]:
survey.add_leg(scanner_settings=scanner_settings, x=0, y=0, z=0)

In [ ]:
points, trajectories = survey.run()

As the definition of legs often requires varying only a few parameters, while keeping many others, the original Helios API discussion contained also ideas of adding interfaces to add multiple legs with caried parameters. We propose a simpler interface that leverages the availability of a high level programming language to resolve the parameter variability and then just use the `add_leg` call in a loop. The utility function `helios.combine_parameters` implements the expansion of parameters:

In [ ]:
for params in helios.combine_parameters(x=[0, 1]):
    print(params)

If you give multiple parameters, by default you build the product space:

In [ ]:
for params in helios.combine_parameters(x=[0, 1], y=[0, 1]):
    print(params)

However, you can control via the `groups` parameter which parameter ranges should be zipped together, instead of building a product:

In [ ]:
for params in helios.combine_parameters(
    x=[0, 1], y=[0, 1], z=[10, 20], groups=[["x", "y"], ["z"]]
):
    print(params)

In fact the function even allows us to define string parameters that use Python format string expressions. They will be formatted against the expanded parameters:

In [ ]:
for params in helios.combine_parameters(x=[0, 1], coordinate="x={x}"):
    print(params)

We hope that this utility function will be useful in a lot of contexts that involve variability of parameters and allows us to keep parameter variability out of the actual interfaces.

**TODOs aus Use Case 2**:
* Trajectory Input for ALS (currently implemented)

## Run survey multiple times with varied settings (Use Case 3)

This will mostly be done via the above `combine_parameters` function and the power of the Python programming language.

## Construct scenes in Python (Use Case 4)

The `ScenePart` object behaves like the other objects we have seen before. However, we do not only have a construction method `from_xml`, but also `from_obj`:

In [ ]:
part = helios.ScenePart.from_obj("data/sceneparts/basic/box/box.obj")

Parts can then be composed into scenes:

In [ ]:
scene = helios.StaticScene(scene_parts=[part])

If you want to apply transformations to a scene part you can chain them together and they will be performed in exactly the given order (this is called *fluent programming*):

In [ ]:
part = (
    helios.ScenePart.from_obj("data/sceneparts/basic/box/box.obj")
    .scale(2.0)
    .translate(np.array([1.0, 0.0, 0.0]))
    .rotate(origin=np.array([1.0, 0.0, 0.0]), image=np.array([0.0, 1.0, 0.0]))
)

Note that rotations can be defined in 3 different ways: Quaternions, Axis + Angle, Origin + Image vector.

**TODOs aus Use Case 4**:
* Add `from_tiff`
* Add `from_xyz`
* Add `from_o3d`

## Change scenes between survey runs (Use Case 5)

We have not experimented with this yet. I expect this to be more a problem of defining precise semantics and asserting them than anything else. In general, we would try to offload as much as possible to the power of the Python programming language. E.g. what is currently done with the *swap on repeat* feature, I can imagine is much easier implemented by programmatically enabling/disabling/transforming scene parts.

## Custom Scanners (Use Case 6)

We have not currently done this due to a lack of priority. I think if we only talk about exposing the configuration for scanners, then this should be absolutely possible with reasonable effort. In contrast, allowing new optics types to be added would be **much** more difficult and also destroy performance - so I would opt out of implementing this.

## Helios Live (Use Case 7)

We did not implement this yet, but the bigger feature support this would be custom callbacks that can be passed to `Survey.run`.

## Dynamic Scenes (Use Case 8)

We have not thought about these yet. My guess is that similar to how `SceneParts` are transformed with fluent programming, their motion should also be recorded in such a way. It is quite likely that this interface will very new.

## Things that were not mentioned in Use Cases

We will look into *serialization* soon - allowing data to be dumped into new file formats. The general idea would be that classes receive methods `to_dict`, `to_yaml`, `to_json` etc. and their `from_*` counterparts. This will hopefully allow us a controlled move from XML to newer filer formats. We would start versioning these new file formats from day 1. XML would stick around as an input source for a while, but would not receive updates when the new format advances.